<a href="https://colab.research.google.com/github/karnjj/sed-young-wa/blob/main/research-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prepare folder
!mkdir pipeline_config
!mkdir source
!mkdir result

In [ ]:
import os 
# careful try/except at `load traffy` and  `create weather` part 
CONFIG_DIR = './pipeline_config'
DATA_DIR = './source'
RESULT_DIR = './result'

# training config
# number of day for scraping weather 
DAY_OF_HIST_WEATHER = 40 

# the year of holiday data.
YEAR_IN_THAI = 2566

# using data for training between [0, current_day - TIME_OFFSET]
TIME_OFFSET = 4 

# Initial Spark

In [ ]:
# For Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!mv spark-3.3.2-bin-hadoop3 spark
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

# [Resource] Load latest data and resources
Load the data from traffy fondue api

## Load Traffy fondue api

In [ ]:
!wget https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv -P $DATA_DIR

--2023-05-18 15:42:33--  https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv
Resolving publicapi.traffy.in.th (publicapi.traffy.in.th)... 35.201.64.130
Connecting to publicapi.traffy.in.th (publicapi.traffy.in.th)|35.201.64.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361890441 (345M) [text/csv]
Saving to: ‘./source/bangkok_traffy.csv’

bangkok_traffy.csv  100%[===================>] 345.12M  21.9MB/s    in 16s     

2023-05-18 15:42:50 (21.4 MB/s) - ‘./source/bangkok_traffy.csv’ saved [361890441/361890441]



## Weather scraping
output `df_weather` which contrain historical weather data.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import pandas as pd
from tqdm import tqdm
import pytz
import time as os_time
import random 
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None

def get_api_key():
  url = requests.get("https://www.wunderground.com/weather/th/bangkok/VTBD")
  soup = BeautifulSoup(url.content, 'html.parser')

  soup = soup.find("script", {"id":"app-root-state"}).text

  start_idx = soup.find("apiKey=")
  end_idx = soup.find("&a", start_idx)

  api_key = soup[start_idx: end_idx].replace("apiKey=", "")

  return api_key

def fill_missing(df):
  if len(df) != 48:
    periods = ['00:00', '00:30', '01:00', '01:30', '02:00', '02:30',
                '03:00', '03:30', '04:00', '04:30', '05:00', '05:30',
                '06:00', '06:30', '07:00', '07:30', '08:00', '08:30',
                '09:00', '09:30', '10:00', '10:30', '11:00', '11:30',
                '12:00', '12:30', '13:00', '13:30', '14:00', '14:30',
                '15:00', '15:30', '16:00', '16:30', '17:00', '17:30',
                '18:00', '18:30', '19:00', '19:30', '20:00', '20:30',
                '21:00', '21:30', '22:00', '22:30', '23:00', '23:30']
    for period in periods:
        if period not in df['time'].values:
            df = df.append({'time': period}, ignore_index=True)
    df = df.sort_values(by=['time'])
    df = df.fillna(method='ffill')
    df = df.drop_duplicates(subset=['time'], keep='first')
    df = df.reset_index(drop=True)
  return df

def scrap_weather_v2(n_day):

  api_key = get_api_key()

  all_df = pd.DataFrame()

  today = datetime.now()
  for i in tqdm(range(1, n_day + 1)):
    df = pd.DataFrame(columns=["date","time","temp (F)","feel_like (F)","dew_point (F)","humidity (%)","wind","wind_speed (mph)","wind_gust (mph)","pressure (in)","precip (in)","condition"])
    target_date = today - timedelta(days=i)
    historical_data = requests.get("https://api.weather.com/v1/location/VTBD:9:TH/observations/historical.json",
                                  params={
                                      "apiKey": api_key,
                                      "units": "e",
                                      "startDate": target_date.strftime("%Y%m%d")
                                  }
                                  ).json()                
    historical_data['observations']
    for data in historical_data['observations']:
      date_time = datetime.fromtimestamp(data['valid_time_gmt'], pytz.timezone("Asia/Bangkok"))
      date = date_time.strftime('%Y-%m-%d')
      time = date_time.strftime('%H:%M')
      new_row = {
          "date": date,
          "time": time,
          "temp (F)": data["temp"],
          "feel_like (F)": data["feels_like"],
          "dew_point (F)": data["dewPt"],
          "humidity (%)": data["rh"],
          "wind": data["wdir_cardinal"],
          "wind_speed (mph)": data["wspd"] or 0,
          "wind_gust (mph)": data["gust"] or 0,
          "pressure (in)": data["pressure"],
          "precip (in)": data["precip_hrly"] or 0,
          "condition": data["wx_phrase"],
      }

      df = df.append(new_row, ignore_index=True)

    df = df.dropna()
    df = fill_missing(df)

    all_df = all_df.append(df, ignore_index=True)

    os_time.sleep(random.randint(1, 3))

  return all_df

In [ ]:
from tqdm import tqdm
from datetime import  datetime, timedelta

try:
  df_weather  
except :
  df_weather = scrap_weather_v2(n_day=DAY_OF_HIST_WEATHER)

# concat datetime
def concat_datetime(data) :
  date = data['date'] + " " + data['time']
  return datetime.strptime(date, "%Y-%m-%d %H:%M")

df_weather['datetime'] = df_weather.apply(concat_datetime, axis=1)
df_weather.head()
df_weather.to_csv(os.path.join(DATA_DIR, "weather_data"))

100%|██████████| 40/40 [02:28<00:00,  3.71s/it]


## Holidays in Thailand (current Year)
output `holidays` which contrains a list of holiday in date format.

In [ ]:
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip -q install dateparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 6.6 MB/s eta 0:00:00


In [ ]:
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup
import dateparser
from datetime import datetime
import json 

# Get current year
url = f"https://calendar.kapook.com/{YEAR_IN_THAI}/holiday" 

url = requests.get(url)
soup = BeautifulSoup(url.content, 'html.parser')

soup = soup.find('div', {"id": "holiday_wrap"}).find_all('span', {"class": "date"})
holidays = set()
for x in soup :
  x = x.text
  dt = dateparser.parse(x)
  dt = datetime(dt.year - 543, dt.month, dt.day)
  holidays.add(dt.strftime("%d/%m/%Y"))

thai_holidays = {"days": list(holidays)}

with open(os.path.join(DATA_DIR,"thai_holidays.json"), "w") as outfile:
  json_object = json.dumps(thai_holidays, indent = 4, ensure_ascii=False) 
  print(json_object)
  outfile.write(json_object)

{
    "days": [
        "10/12/2023",
        "02/01/2023",
        "23/10/2023",
        "13/04/2023",
        "14/08/2023",
        "02/08/2023",
        "17/04/2023",
        "01/05/2023",
        "05/12/2023",
        "31/12/2023",
        "01/08/2023",
        "29/10/2023",
        "17/05/2023",
        "14/04/2023",
        "06/04/2023",
        "05/06/2023",
        "28/07/2023",
        "15/04/2023",
        "01/01/2023",
        "13/10/2023",
        "03/06/2023",
        "12/08/2023",
        "04/05/2023",
        "11/12/2023",
        "06/03/2023",
        "05/05/2023"
    ]
}


# Data Preparation

## Load necessary data

In [ ]:
# load traffy fondue
from pyspark.sql import functions as F
from pyspark.sql.types import *

path = os.path.join(DATA_DIR, "bangkok_traffy.csv")

spark_df = spark.read.csv(path, header=True, inferSchema=True, sep=',', escape="\"", encoding='utf-8', multiLine=True)

# For develop
try :
  df_raw
except:
  df_raw = spark_df.toPandas()

df_raw.tail(5).to_csv(os.path.join(CONFIG_DIR, "sample_query.csv"))
df_raw.head(2)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-9LHDM6,{},None,ไม่มีภาพ,https://storage.googleapis.com/traffy_public_b...,None,"100.48661,13.79386",1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุง...,บางพลัด,บางพลัด,กรุงเทพมหานคร,2021-09-01 10:44:55.353209,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:59:58.622268
1,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,None,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,None,None,กรุงเทพมหานคร,2021-09-03 12:51:09.453003,เสร็จสิ้น,NaN,NaN,2022-06-04 15:34:14.609206


In [ ]:
# Load Thai holiday
with open(os.path.join(DATA_DIR,"thai_holidays.json"), "r") as f:
  holidays = json.load(f)['days']

# Load weather data
df_weather = pd.read_csv(os.path.join(DATA_DIR, "weather_data"), parse_dates=['datetime'])

In [ ]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unnamed: 0        1920 non-null   int64         
 1   date              1920 non-null   object        
 2   time              1920 non-null   object        
 3   temp (F)          1920 non-null   int64         
 4   feel_like (F)     1920 non-null   int64         
 5   dew_point (F)     1920 non-null   int64         
 6   humidity (%)      1920 non-null   int64         
 7   wind              1920 non-null   object        
 8   wind_speed (mph)  1920 non-null   int64         
 9   wind_gust (mph)   1920 non-null   int64         
 10  pressure (in)     1920 non-null   float64       
 11  precip (in)       1920 non-null   int64         
 12  condition         1920 non-null   object        
 13  datetime          1920 non-null   datetime64[ns]
dtypes: datetime64[ns](1), fl

## Set mark day
- Set current_day
- Set end_date_train

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

current_day = datetime.now()
# current_day = datetime(2023,5,12)
end_date_train = current_day - timedelta(days=TIME_OFFSET)
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
df = df_raw[df_raw['timestamp'] <= end_date_train]
max(df['timestamp'])

Timestamp('2023-05-14 15:44:54.297808')

In [ ]:
df_raw.head(2)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-9LHDM6,{},None,ไม่มีภาพ,https://storage.googleapis.com/traffy_public_b...,None,"100.48661,13.79386",1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุง...,บางพลัด,บางพลัด,กรุงเทพมหานคร,2021-09-01 10:44:55.353209,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:59:58.622268
1,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,None,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,None,None,กรุงเทพมหานคร,2021-09-03 12:51:09.453003,เสร็จสิ้น,NaN,NaN,2022-06-04 15:34:14.609206


## Drop unused columns
Select only necessary columns

In [ ]:
target_columns = [ "ticket_id", "type", "district", "timestamp", "last_activity"]
df = df[~df['ticket_id'].isna()]
df = df[df['state'] == "เสร็จสิ้น"]
df = df[target_columns]

print(df.shape)
df.head()

(202154, 5)


,ticket_id,type,district,timestamp,last_activity
1,2021-FYJTFP,{ความสะอาด},None,2021-09-03 12:51:09.453003,2022-06-04 15:34:14.609206
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,2022-06-21 08:21:09.532782
10,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,2022-06-06 01:17:12.272904
12,2021-9U2NJT,{น้ำท่วม},None,2021-10-14 10:45:27.713884,2022-09-08 08:35:43.784519
25,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,2022-08-12 07:18:44.884945


## Data cleaning


### Format date


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['last_activity'] = pd.to_datetime(df['last_activity'])

### Deal with NULL value
- Fill type with `{}`
- Drop out the data which have missing of `district`. 

In [ ]:
df.isna().sum()

ticket_id         0
type             96
district         47
timestamp         0
last_activity     0
dtype: int64

In [ ]:
df['type'].fillna("{}", inplace=True)
df.dropna(subset=['district'], inplace=True)

## Feature enginearing
- Create a feature `time_of_day` which is the unit of a day in between 0-23.
- Create type features which are the group of high frequency on the data.

In [ ]:
selected_feature_value = {"types":[], "provinces":[]}

### Hours finish
- Create the `process_hour` from `last_activity` minus `timestamp`

In [ ]:
def get_process_hour(data) :
  dt = data['last_activity'] - data['timestamp']
  total_hour = dt.days * 24 + dt.seconds//3600
  return total_hour
df['process_hour'] = df.apply(get_process_hour, axis=1)
df.drop(["last_activity"], axis=1, inplace = True)
df.head()

,ticket_id,type,district,timestamp,process_hour
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,6593
10,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,6068
25,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,5898
64,2021-4D9Y98,{},ลาดพร้าว,2021-12-13 05:53:36.861064,10950
102,2021-7U9RED,{},ดุสิต,2021-12-17 08:46:02.610983,12381


### Time of a day

In [ ]:
df['time_of_day'] = df['timestamp'].dt.hour

### Type
- Filter with the frequency of each category.

In [ ]:
#@title Fillter high cardinality
from mpl_toolkits.mplot3d.axes3d import defaultdict

type_freq = defaultdict(int)
get_by = "rank" #@param ["ratio", "rank"]
k = 7 #@param 
ratio_value = 0.5 #@param 


def get_types(data):
  data = data.strip()
  data = data[1:-1] # trim "{}"
  types = data.split(",")
  return types

# reset index
df = df.reset_index(drop=True)

# get set of type
for i in range(len(df)) :
  data = get_types(df['type'][i])
  for d in data :
      type_freq[d]+=1
type_freq.pop("")
type_freq = sorted([(v/df.shape[0], k) for k,v in type_freq.items()], reverse=True)
type_freq

if get_by == "ratio" :
  target_type = []
  for x in type_freq :
    if x[0] > ratio_value :
      target_type.append(x[1])
elif get_by == "rank" :
  target_type = [x[1] for x in type_freq[:k]]

# create type feature
for t in target_type :
  df[t] = df['type'].map(lambda x: 1 if t in get_types(x) else 0)

# df.drop(columns=['type'], inplace=True)
print(target_type)
selected_feature_value['types'] = target_type
df.head()

['ถนน', 'ทางเท้า', 'ความสะอาด', 'น้ำท่วม', 'ความปลอดภัย', 'กีดขวาง', 'แสงสว่าง']


,ticket_id,type,district,timestamp,process_hour,time_of_day,ถนน,ทางเท้า,ความสะอาด,น้ำท่วม,ความปลอดภัย,กีดขวาง,แสงสว่าง
0,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,6593,14,0,0,0,1,0,0,0
1,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,6068,5,0,0,0,0,0,0,0
2,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,5898,12,1,0,0,1,0,0,0
3,2021-4D9Y98,{},ลาดพร้าว,2021-12-13 05:53:36.861064,10950,5,0,0,0,0,0,0,0
4,2021-7U9RED,{},ดุสิต,2021-12-17 08:46:02.610983,12381,8,0,0,0,0,0,0,0


### District
- Filter with the frequency of each category.

In [ ]:
df.head()

,ticket_id,type,district,timestamp,process_hour,time_of_day,ถนน,ทางเท้า,ความสะอาด,น้ำท่วม,ความปลอดภัย,กีดขวาง,แสงสว่าง
0,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,6593,14,0,0,0,1,0,0,0
1,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,6068,5,0,0,0,0,0,0,0
2,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,5898,12,1,0,0,1,0,0,0
3,2021-4D9Y98,{},ลาดพร้าว,2021-12-13 05:53:36.861064,10950,5,0,0,0,0,0,0,0
4,2021-7U9RED,{},ดุสิต,2021-12-17 08:46:02.610983,12381,8,0,0,0,0,0,0,0


In [ ]:
#@title Fillter high cardinality

get_by = "rank" #@param ["ratio", "rank"]
k = 25 #@param 
ratio_value = 0.5 #@param 

district_freq = df.groupby("district")["district"].count()/df.shape[0]
district_freq = sorted([(x[1],x[0]) for x in district_freq.items()], reverse=True)
district_freq

if get_by == "ratio" :
  target_provinces = []
  for x in district_freq :
    if x[0] > ratio_value :
      target_provinces.append(x[1])
elif get_by == "rank" :
  target_provinces = [x[1] for x in district_freq[:k]]

if "other" not in target_provinces :
  target_provinces += ["other"]

print(target_provinces)
selected_feature_value['provinces'] = target_provinces
# map with target_provices
df['district'] = df['district'].map(lambda x: x if x in target_provinces else "other")


# create
for province in target_provinces :
  df[province] = df['district'].apply(lambda x: 1 if x == province else 0)

df.head()

['จตุจักร', 'ประเวศ', 'บางแค', 'บางเขน', 'ลาดกระบัง', 'บางกะปิ', 'วัฒนา', 'คลองเตย', 'สวนหลวง', 'บางขุนเทียน', 'บึงกุ่ม', 'บางซื่อ', 'ราชเทวี', 'ปทุมวัน', 'ดินแดง', 'บางกอกน้อย', 'ลาดพร้าว', 'สาทร', 'บางรัก', 'วังทองหลาง', 'พญาไท', 'ธนบุรี', 'สายไหม', 'บางนา', 'คลองสามวา', 'other']


,ticket_id,type,district,timestamp,process_hour,time_of_day,ถนน,ทางเท้า,ความสะอาด,น้ำท่วม,...,ลาดพร้าว,สาทร,บางรัก,วังทองหลาง,พญาไท,ธนบุรี,สายไหม,บางนา,คลองสามวา,other
0,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,6593,14,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,6068,5,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,5898,12,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,2021-4D9Y98,{},ลาดพร้าว,2021-12-13 05:53:36.861064,10950,5,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2021-7U9RED,{},other,2021-12-17 08:46:02.610983,12381,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Create holiday holiday features

In [ ]:
def is_holiday_offset(data, offset=0) :
  day = data + timedelta(days=offset)
  day_text = day.strftime("%d/%m/%Y")
  if (day_text in holidays) or (day.day_name() in ['Saturday', 'Sunday']) :
    return 1
  else: return 0

look_ahead = 5 # days
for l in range(look_ahead) :
  df['is_holiday_'+str(l)] = df['timestamp'].map(lambda x: is_holiday_offset(x, l))

## Create weather information

In [ ]:
weather_conditions = ["condition_" + x for x in df_weather['condition'].unique()]
df_weather_d = df_weather.drop(columns=['date', "time", "wind", "wind_speed (mph)", "wind_gust (mph)", "pressure (in)", "precip (in)", "dew_point (F)"])
df_weather_d = pd.get_dummies(df_weather_d, columns=["condition"])
df_weather_d.head(1)

,Unnamed: 0,temp (F),feel_like (F),humidity (%),datetime,condition_Fair,condition_Light Rain,condition_Light Rain / Windy,condition_Light Rain with Thunder,condition_Mostly Cloudy,condition_Mostly Cloudy / Windy,condition_Partly Cloudy,condition_Rain,condition_T-Storm / Windy,condition_Thunder
0,0,88,100,70,2023-05-17,1,0,0,0,0,0,0,0,0,0


In [ ]:
# mark only degree of 3 ex [3,9,15,21]
mark_number = [3,9,15,21]
mark_data = df_weather_d['datetime'].map(lambda x: ((x.hour in mark_number) and (x.minute == 0)))
df_weather_mark = df_weather_d[mark_data]
df_weather_mark.sort_values("datetime", inplace=True) 
df_weather_mark.reset_index(drop=True, inplace=True)

In [ ]:
import numpy as np

def nearest_ind(items, pivot):
  # assume items are sorted
  l, r = 0, len(items)-1
  while l < r :
    mid = (l+r)//2
    d = (items[mid] - pivot).total_seconds()
    if d > 0 :
      r = mid 
    else :
      l = mid+1
  # print(items[l], pivot)
  return l

# parameters
w_lookhead_k = 3*4
weather_features = ["temp (F)", "humidity (%)"] 

dict_features = {}
for i in range(w_lookhead_k) :
  for w in weather_features :
    dict_features[str(i)+ "_" + w] = []

missing = 0
all = 0

for target_timestamp in tqdm(df['timestamp']) :
  ind_target_time = nearest_ind(df_weather_mark['datetime'], target_timestamp) + 1
  # ind_target_time = 0
  for i in range(w_lookhead_k) :
    for w in weather_features :
      all += 1
      if ind_target_time+i >= len(df_weather_mark) :
        missing+=1
        dict_features[str(i)+ "_" + w].append(dict_features[str(i-1)+ "_" + w][-1])
      else :
        dict_features[str(i)+ "_" + w].append(df_weather_mark[w][ind_target_time+i])
for k,v in dict_features.items() :
  df[k] = v
print("missing weather:",missing/all)
df.head()

100%|██████████| 202107/202107 [01:07<00:00, 2974.05it/s]


missing weather: 0.0


,ticket_id,type,district,timestamp,process_hour,time_of_day,ถนน,ทางเท้า,ความสะอาด,น้ำท่วม,...,7_temp (F),7_humidity (%),8_temp (F),8_humidity (%),9_temp (F),9_humidity (%),10_temp (F),10_humidity (%),11_temp (F),11_humidity (%)
0,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ประเวศ,2021-09-19 14:56:08.924992,6593,14,0,0,0,1,...,84,79,88,75,97,50,86,74,86,74
1,2021-7XATFA,{สะพาน},สาทร,2021-09-26 05:03:52.594898,6068,5,0,0,0,0,...,84,79,88,75,97,50,86,74,86,74
2,2021-DVEWYM,"{น้ำท่วม,ถนน}",ลาดพร้าว,2021-12-09 12:29:08.408763,5898,12,1,0,0,1,...,84,79,88,75,97,50,86,74,86,74
3,2021-4D9Y98,{},ลาดพร้าว,2021-12-13 05:53:36.861064,10950,5,0,0,0,0,...,84,79,88,75,97,50,86,74,86,74
4,2021-7U9RED,{},other,2021-12-17 08:46:02.610983,12381,8,0,0,0,0,...,84,79,88,75,97,50,86,74,86,74


In [ ]:
df_after_weather_keep = df.copy

## [Save] selected_feature_value

In [ ]:
import json 
with open(os.path.join(CONFIG_DIR,"selected_feature_value.json"), "w") as outfile:
  json_object = json.dumps(selected_feature_value, indent = 4, ensure_ascii=False) 
  print(json_object)
  outfile.write(json_object)

{
    "types": [
        "ถนน",
        "ทางเท้า",
        "ความสะอาด",
        "น้ำท่วม",
        "ความปลอดภัย",
        "กีดขวาง",
        "แสงสว่าง"
    ],
    "provinces": [
        "จตุจักร",
        "ประเวศ",
        "บางแค",
        "บางเขน",
        "ลาดกระบัง",
        "บางกะปิ",
        "วัฒนา",
        "คลองเตย",
        "สวนหลวง",
        "บางขุนเทียน",
        "บึงกุ่ม",
        "บางซื่อ",
        "ราชเทวี",
        "ปทุมวัน",
        "ดินแดง",
        "บางกอกน้อย",
        "ลาดพร้าว",
        "สาทร",
        "บางรัก",
        "วังทองหลาง",
        "พญาไท",
        "ธนบุรี",
        "สายไหม",
        "บางนา",
        "คลองสามวา",
        "other"
    ]
}


# Defind period of time

```
df_test = [end_date_train - k_test,end_date_train]
df_train = [end_date_train - k_test - k_train, end_date_train - k_test ]
```

In [ ]:
#@title define start_train, end_train, end_test time
k_test = 7 #@param {type:"integer"}
k_train = 25 #@param {type:"integer"}

from datetime import  datetime, timedelta

# current_date = datetime.today()
current_date = end_date_train
start_test_date = end_date_train - timedelta(days=k_test)
start_train_date = end_date_train- timedelta(days=k_test) - timedelta(days=k_train)

print("train on ", start_train_date, " to ",start_test_date)
print("test on ", start_test_date, " to ",current_date)

# filter data
df_test = df[df['timestamp'] > start_test_date]
df_train = df[( df['timestamp'] > start_train_date ) & ( df['timestamp'] < start_test_date )]
total_df = df_test.shape[0] + df_train.shape[0]
print(f"test set: {df_test.shape[0]}\n train set: {df_train.shape[0]}\n")
print(f"test set: {df_test.shape[0]/total_df}\n train set: {df_train.shape[0]/total_df}")

train on  2023-04-12 15:46:37.097891  to  2023-05-07 15:46:37.097891
test on  2023-05-07 15:46:37.097891  to  2023-05-14 15:46:37.097891
test set: 1358
 train set: 7508

test set: 0.15316941123392736
 train set: 0.8468305887660726


# Prepare Features

In [ ]:
df_train.head()

,ticket_id,type,district,timestamp,process_hour,time_of_day,ถนน,ทางเท้า,ความสะอาด,น้ำท่วม,...,7_temp (F),7_humidity (%),8_temp (F),8_humidity (%),9_temp (F),9_humidity (%),10_temp (F),10_humidity (%),11_temp (F),11_humidity (%)
193241,2023-3HGUVD,{ต้นไม้},สวนหลวง,2023-04-12 15:47:59.694765,35,15,0,0,0,0,...,90,52,84,84,90,70,99,50,90,62
193242,2023-N7V32D,{ทางเท้า},other,2023-04-12 16:02:16.907755,254,16,0,1,0,0,...,90,52,84,84,90,70,99,50,90,62
193243,2023-3TGFQV,{ทางเท้า},other,2023-04-12 16:08:31.399695,640,16,0,1,0,0,...,90,52,84,84,90,70,99,50,90,62
193244,2023-6NLQ44,{เสียงรบกวน},other,2023-04-12 16:41:26.914410,134,16,0,0,0,0,...,90,52,84,84,90,70,99,50,90,62
193245,2023-D9XWGN,{เสียงรบกวน},other,2023-04-12 16:45:09.933172,134,16,0,0,0,0,...,90,52,84,84,90,70,99,50,90,62


In [ ]:

X_train = df_train.iloc[:,len(target_columns):]
y_train = df_train["process_hour"]

X_test = df_test.iloc[:,len(target_columns):]
y_test = df_test["process_hour"]
X_train.shape, X_test.shape

((7508, 63), (1358, 63))

In [ ]:
#@title [Save] standardScaler
# normalize feature
from sklearn.preprocessing import StandardScaler
import pickle
import json 

scaler = StandardScaler()
X_train[:] = scaler.fit_transform(X_train)
X_test[:] = scaler.transform(X_test)

check_order=  {"order": list(X_train.columns)}

with open(os.path.join(CONFIG_DIR,"order_check.json"), "w") as outfile:
  json_object = json.dumps(check_order, indent = 4, ensure_ascii=False) 
  outfile.write(json_object)


# dump standard 
file_name = os.path.join(CONFIG_DIR, "scalar.pkl")
with open(file_name,'wb') as f:
    pickle.dump(scaler, f)

# Feature selection

## Mutual_info_regression

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
  mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
  mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
  mi_scores = mi_scores.sort_values(ascending=False)
  return mi_scores 


X_train = X_train.astype(int)
discrete_features = X_train.dtypes == int

mi_scores = make_mi_scores(X_train, y_train, discrete_features)
for col, v in mi_scores.items() :
  print(col, v)

time_of_day 0.2639732257331706
1_humidity (%) 0.13981410229023972
3_humidity (%) 0.1396161631116093
3_temp (F) 0.13646378786510338
11_humidity (%) 0.13200994330881888
7_humidity (%) 0.128650125862958
0_temp (F) 0.12571329004639797
5_humidity (%) 0.12226991716247682
4_humidity (%) 0.1182127478342303
1_temp (F) 0.117857108473578
0_humidity (%) 0.11764556596513365
11_temp (F) 0.11640364449355611
9_humidity (%) 0.11512460629042076
7_temp (F) 0.10529131169130701
8_humidity (%) 0.10311683941751237
5_temp (F) 0.10307379733907895
6_temp (F) 0.09903207609510956
6_humidity (%) 0.09817869126494316
9_temp (F) 0.09538349678691938
2_humidity (%) 0.09215308320961912
4_temp (F) 0.09144458396498756
8_temp (F) 0.0903965723398148
2_temp (F) 0.07593246139186705
10_temp (F) 0.07303199012119244
10_humidity (%) 0.07107849839241509
is_holiday_1 0.05586553358221469
is_holiday_2 0.04289298202021263
บางแค 0.041009556162168614
is_holiday_3 0.036785583205851013
is_holiday_0 0.036126308481559954
is_holiday_4 0.0356

## Top K mutual_info_reg

In [ ]:
# from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
# slk =  SelectKBest(mutual_info_regression, k=30)
# X_new_train = slk.fit_transform(X_train, y_train)
# X_new_test = slk.transform(X_test)
# # X_new_valid = slk.transform(X_valid)
# X_train.columns[slk.get_support()]

## Model select base

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              AdaBoostRegressor)

selection_model = GradientBoostingRegressor(learning_rate=0.2, max_depth=7, min_samples_split=350,
                          n_estimators=50, random_state=1).fit(X_train, y_train)
model_feature = SelectFromModel(selection_model, prefit=True)
X_new_train = model_feature.transform(X_train)
X_new_test = model_feature.transform(X_test)
X_new_train.shape, X_train.shape

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


((7508, 20), (7508, 63))

In [ ]:
X_train.columns[model_feature.get_support()]

Index(['time_of_day', 'ความสะอาด', 'ความปลอดภัย', 'แสงสว่าง', 'จตุจักร',
       'บางแค', 'ลาดกระบัง', 'บางกะปิ', 'คลองเตย', 'ปทุมวัน', 'ลาดพร้าว',
       'บางรัก', 'บางนา', 'other', 'is_holiday_1', 'is_holiday_2',
       '0_humidity (%)', '2_humidity (%)', '3_humidity (%)', '4_humidity (%)'],
      dtype='object')

# Model

## Model selection

In [ ]:
#We will save the model performance metrics in a DataFrame
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              AdaBoostRegressor)
from xgboost import XGBRegressor
import numpy as np

Model = []
RMSE = []
R_sq = []
cv = KFold(5, random_state = 1, shuffle=True)

names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
         'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor',
         'Adaboost Regressor', "XGBRegressor"]
models = [LinearRegression(), Ridge(), Lasso(),
          KNeighborsRegressor(), DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(), 
          AdaBoostRegressor(), XGBRegressor()]

#Creating a Function to append the cross validation scores of the algorithms
def input_scores(name, model, x, y):
    Model.append(name)
    RMSE.append(np.sqrt((-1) * cross_val_score(model, x, y, cv=cv, 
                                               scoring='neg_mean_squared_error').mean()))
    R_sq.append(cross_val_score(model, x, y, cv=cv, scoring='r2').mean())

#Running all algorithms
for name, model in zip(names, models):
    input_scores(name, model, X_new_train, y_train)

evaluation = pd.DataFrame({'Model': Model,
                           'RMSE': RMSE,
                           'R Squared': R_sq})
print("FOLLOWING ARE THE TRAINING SCORES: ")
evaluation

## Model tunning

In [ ]:
#tuning for number of trees
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(5, random_state = 1, shuffle=True)
param_grid = {'n_estimators': [50],
              'max_depth': [7], #range(5,16,2), 
              'min_samples_split': range(100,1001,200), #range(100,1001,200), 
              'learning_rate':[0.2]}
clf = GridSearchCV(GradientBoostingRegressor(random_state=1), 
                   param_grid = param_grid, scoring='r2', 
                   cv=cv).fit(X_train, y_train)

best_model = clf.best_estimator_
print(best_model) 
print("R Squared:",clf.best_score_)

In [ ]:
# Test dataset
import numpy as np
r2_test = cross_val_score(best_model, X_test, y_test, cv=cv, scoring='r2').mean()
RMSE_test = np.sqrt((-1) * cross_val_score(best_model, X_test, y_test, cv=cv, scoring='neg_mean_squared_error').mean())
print(" == Test set ==")
print("R Seuared of test set: %.2f" % r2_test)
print("RMSE of test set: %.2f" % RMSE_test)

# [Save] Model

In [ ]:
import pickle
filename = os.path.join(CONFIG_DIR,'model_satee.sav')
pickle.dump(best_model, open(filename, 'wb'))

# [Save] Prepare the data for visualize

In [ ]:
# Get result
X_v = df.iloc[:, 5:]
df_visual = df.copy()
df_visual['estimated_hour'] = best_model.predict(X_v)
df_visual.drop(columns=['type', 'district', 'timestamp'], inplace=True)

In [ ]:
out_df_visual = df_raw.merge(df_visual, how="left", on="ticket_id")
out_df_visual.to_csv(os.path.join(RESULT_DIR, "df_visual.csv"))
out_df_visual.head()

In [ ]:
out_df_visual.columns